In [1]:
import pandas as pd
import numpy as np

In [2]:
loans = pd.read_csv('../data/lending-club-data.csv')

/home/om/gl-env/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : 1 if x == 0 else -1)
loans = loans.drop(['bad_loans'],axis = 1)

In [4]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'

In [6]:
loans = loans[features + [target]]

In [11]:
string = []
for feature in features:
    if loans[feature].dtypes == "object":
        string.append(feature)
print string

['grade', 'term', 'home_ownership', 'emp_length']


In [12]:
for x in string:
    for y in loans[x].unique():
        loans[y] = loans[x].apply(lambda z : 1 if z == y else 0)

In [14]:
loans = loans.drop(string,axis =1)

In [15]:
train_id = list(pd.read_json('../data/module-6-assignment-train-idx.json')[0])
validation_id = list(pd.read_json('../data/module-6-assignment-validation-idx.json')[0])

In [16]:
train_data = loans.iloc[train_id]
validation_data = loans.iloc[validation_id]
print "Train data : ",len(train_data)
print "validation data : ",len(validation_data)

Train data :  37224
validation data :  9284


# A function to deal with the condition 2 of early stopping

In [19]:
def reached_minimum_node_size(data,min_node_size):
    return len(data) <= min_node_size

# Quiz 1)

## It should stop there

### A function to deal with condition 3 of early stopping

In [21]:
def error_reduction(error_before_split,error_after_split):
    return error_before_split-error_after_split

# Quiz 2)

## Goes with 0.14

In [22]:
def intermediate_node_num_mistakes(labels_in_node):
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0
    
    # Count the number of 1's (safe loans)
    safe_loans = list(labels_in_node).count(1)
    
    # Count the number of -1's (risky loans)
    risky_loans = list(labels_in_node).count(-1)
                
    # Return the number of mistakes that the majority classifier makes.
    return min(safe_loans,risky_loans)

In [39]:
def best_splitting_feature(data, features, target):
    
    target_values = data[target]
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        # The left split will have all data points where the feature value is 0
        left_split = data[data[feature] == 0]
        
        # The right split will have all data points where the feature value is 1
        ## YOUR CODE HERE
        right_split = data[data[feature] == 1]
            
        # Calculate the number of misclassified examples in the left split.
        # Remember that we implemented a function for this! (It was called intermediate_node_num_mistakes)
        # YOUR CODE HERE
        left_mistakes = intermediate_node_num_mistakes(left_split[target])       

        # Calculate the number of misclassified examples in the right split.
        ## YOUR CODE HERE
        right_mistakes = intermediate_node_num_mistakes(right_split[target])
            
        # Compute the classification error of this split.
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        ## YOUR CODE HERE
        error = (left_mistakes + right_mistakes) / num_data_points

        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        ## YOUR CODE HERE
        if error < best_error:
            best_error = error
            best_feature = feature
            
    
    return best_feature # Return the best feature we found

In [40]:
def create_leaf(target_values):
    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True }   ## YOUR CODE HERE
    
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    
    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] = +1         ## YOUR CODE HERE
    else:
        leaf['prediction'] = -1         ## YOUR CODE HERE
        
    # Return the leaf node        
    return leaf 

In [54]:
def decision_tree_create(data, features, target, current_depth = 0, 
                         max_depth = 10, min_node_size=1, 
                         min_error_reduction=0.0):
    
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    print "--------------------------------------------------------------------"
    print "Subtree, depth = %s (%s data points)." % (current_depth, len(target_values))
    
    
    # Stopping condition 1: All nodes are of the same type.
    if intermediate_node_num_mistakes(target_values) == 0:
        print "Stopping condition 1 reached. All data points have the same target value."                
        return create_leaf(target_values)
    
    # Stopping condition 2: No more features to split on.
    if remaining_features == []:
        print "Stopping condition 2 reached. No remaining features."                
        return create_leaf(target_values)    
    
    # Early stopping condition 1: Reached max depth limit.
    if current_depth >= max_depth:
        print "Early stopping condition 1 reached. Reached maximum depth."
        return create_leaf(target_values)
    
    # Early stopping condition 2: Reached the minimum node size.
    # If the number of data points is less than or equal to the minimum size, return a leaf.
    if reached_minimum_node_size(target_values,min_node_size) :         ## YOUR CODE HERE 
        print "Early stopping condition 2 reached. Reached minimum node size."
        return create_leaf(target_values)   ## YOUR CODE HERE
    
    # Find the best splitting feature
    splitting_feature = best_splitting_feature(data, features, target)
    
    # Split on the best feature that we found. 
    print splitting_feature
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    
    # Early stopping condition 3: Minimum error reduction
    # Calculate the error before splitting (number of misclassified examples 
    # divided by the total number of examples)
    error_before_split = intermediate_node_num_mistakes(target_values) / float(len(data))
    
    # Calculate the error after splitting (number of misclassified examples 
    # in both groups divided by the total number of examples)
    left_mistakes =    intermediate_node_num_mistakes(left_split)## YOUR CODE HERE
    right_mistakes =   intermediate_node_num_mistakes(right_split)## YOUR CODE HERE
    error_after_split = (left_mistakes + right_mistakes) / float(len(data))
    
    # If the error reduction is LESS THAN OR EQUAL TO min_error_reduction, return a leaf.
    if error_reduction(error_before_split,error_after_split) <= min_error_reduction :        ## YOUR CODE HERE
        print "Early stopping condition 3 reached. Minimum error reduction."
        return   create_leaf(target_values)## YOUR CODE HERE 
    
    
    remaining_features.remove(splitting_feature)
    print "Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split))
    
    
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction)        
    
    ## YOUR CODE HERE
    right_tree = decision_tree_create(right_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction)
    
    
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [55]:

features = list(loans)
features.pop(0)
print features

['B', 'C', 'A', 'E', 'F', 'D', 'G', ' 36 months', ' 60 months', 'RENT', 'OWN', 'MORTGAGE', 'OTHER', '10+ years', '< 1 year', '3 years', '9 years', '4 years', '5 years', '1 year', '6 years', '2 years', '7 years', '8 years', 'n/a']


In [56]:
my_decision_tree_new = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 100, min_error_reduction=0.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
 36 months
Split on feature  36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
A
Split on feature A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
B
Split on feature B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
C
Split on feature C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
E
Split on feature E. (3751, 2133)
--------------------------------------------------------------------
Subtree, depth = 5 (3751 data points).
F
Split on feature F. (2397, 1354)
--------------------------------------------------------------------
Subtree, depth = 6 (2397 data points).
Early stopping 

In [57]:
def classify(tree, x, annotate = False):
       # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate:
             print "At leaf, predicting %s" % tree['prediction']
        return tree['prediction']
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate:
             print "Split on %s = %s" % (tree['splitting_feature'], split_feature_value)
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'],x,annotate)
               ### YOUR CODE

In [58]:
x = validation_data.iloc[0]
print 'Predicted class: %s ' % classify(my_decision_tree_new, x)


Predicted class: -1 


In [59]:
classify(my_decision_tree_new, x, annotate = True)

Split on  36 months = 0.0
Split on A = 0.0
Split on B = 0.0
Split on C = 0.0
Split on E = 0.0
Split on F = 0.0
At leaf, predicting -1


-1

# quiz 3)

## Shorter

# quiz 4)

## Always Shorter

# quiz 5)

## One

In [62]:
def evaluate_classification_error(tree, data):
    # Apply classify(tree, x) to each row in your data
    data['prediction'] = data.apply(lambda x: classify(tree, x),axis = 1)
    error = data[data['prediction'] != data[target]]
    print len(error)
    print len(data)
    return len(error)/float(len(data))
    
    # Once you've made the prediction, calculate the classification error
    ## YOUR CODE HERE

In [63]:
evaluate_classification_error(my_decision_tree_new, validation_data)

3566
9284


/home/om/gl-env/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


0.38410168031021114

In [64]:
model_1 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 2, 
                                min_node_size = 0, min_error_reduction=-1)

model_2 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 0, min_error_reduction=-1)

model_3 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 14, 
                                min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
 36 months
Split on feature  36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
A
Split on feature A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 1 (28001 data points).
D
Split on feature D. (23300, 4701)
--------------------------------------------------------------------
Subtree, depth = 2 (23300 data points).
Early stopping condition 1 reached. Reached maximum depth.
-------------------------------------------------

In [65]:
print "Training data, classification error (model 1):", evaluate_classification_error(model_1, train_data)
print "Training data, classification error (model 2):", evaluate_classification_error(model_2, train_data)
print "Training data, classification error (model 3):", evaluate_classification_error(model_3, train_data)

Training data, classification error (model 1):

/home/om/gl-env/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


 14891
37224
0.400037610144
Training data, classification error (model 2): 14214
37224
0.381850419084
Training data, classification error (model 3): 13938
37224
0.37443584784


In [66]:
print "Validation data, classification error (model 1):", evaluate_classification_error(model_1, validation_data)
print "Validation data, classification error (model 2):", evaluate_classification_error(model_2, validation_data)
print "Validation data, classification error (model 3):", evaluate_classification_error(model_3, validation_data)

Validation data, classification error (model 1):

/home/om/gl-env/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


 3696
9284
0.398104265403
Validation data, classification error (model 2): 3563
9284
0.383778543731
Validation data, classification error (model 3): 3526
9284
0.379793192589


# quiz 6)

## Model 3 has least error on the validation data

# quiz 7)

## Yes

# quiz8)

## No

In [67]:
def count_leaves(tree):
    if tree['is_leaf']:
        return 1
    return count_leaves(tree['left']) + count_leaves(tree['right'])

In [68]:
leaf_model_1 = count_leaves(model_1)
leaf_model_2 = count_leaves(model_2)
leaf_model_3 = count_leaves(model_3)

print "Complexity Model 1 : ",leaf_model_1

print "Complexity Model 2 : ",leaf_model_2

print "Complexity Model 3 : ",leaf_model_3

Complexity Model 1 :  4
Complexity Model 2 :  41
Complexity Model 3 :  361


# Quiz 9)

## Model 3 has largest complexity

# Quiz 10)

## No

In [69]:
model_4 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 0, min_error_reduction=-1)
model_5 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 0, min_error_reduction=0)
model_6 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 0, min_error_reduction=5)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
 36 months
Split on feature  36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
A
Split on feature A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
B
Split on feature B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
C
Split on feature C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
E
Split on feature E. (3751, 2133)
--------------------------------------------------------------------
Subtree, depth = 5 (3751 data points).
F
Split on feature F. (2397, 1354)
--------------------------------------------------------------------
Subtree, depth = 6 (2397 data points).
Early stopping 

In [71]:
print "Validation data, classification error (model 4):", evaluate_classification_error(model_4, validation_data)
print "Validation data, classification error (model 5):", evaluate_classification_error(model_5, validation_data)
print "Validation data, classification error (model 6):", evaluate_classification_error(model_6, validation_data)

 Validation data, classification error (model 4):

/home/om/gl-env/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


 3563
9284
0.383778543731
Validation data, classification error (model 5): 3563
9284
0.383778543731
Validation data, classification error (model 6): 4674
9284
0.503446790177


In [72]:
leaf_model_4 = count_leaves(model_4)
leaf_model_5 = count_leaves(model_5)
leaf_model_6 = count_leaves(model_6)

print "Complexity Model 4 : ",leaf_model_4

print "Complexity Model 5 : ",leaf_model_5

print "Complexity Model 6 : ",leaf_model_6

Complexity Model 4 :  41
Complexity Model 5 :  41
Complexity Model 6 :  1


# Quiz 11)

## Model 4 and Model 5 have largest complexity

In [73]:
model_7 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 0, min_error_reduction=-1)
model_8 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 2000, min_error_reduction=-1)
model_9 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 50000, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
 36 months
Split on feature  36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
A
Split on feature A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
B
Split on feature B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
C
Split on feature C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
E
Split on feature E. (3751, 2133)
--------------------------------------------------------------------
Subtree, depth = 5 (3751 data points).
F
Split on feature F. (2397, 1354)
--------------------------------------------------------------------
Subtree, depth = 6 (2397 data points).
Early stopping 

In [75]:
print "Validation data, classification error (model 7):", 1-evaluate_classification_error(model_7, validation_data)
print "Validation data, classification error (model 8):", 1-evaluate_classification_error(model_8, validation_data)
print "Validation data, classification error (model 9):", 1-evaluate_classification_error(model_9, validation_data)

Validation data, classification error (model 7):

/home/om/gl-env/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


 3563
9284
0.616221456269
Validation data, classification error (model 8): 3570
9284
0.615467470918
Validation data, classification error (model 9): 4674
9284
0.496553209823


In [76]:
leaf_model_7 = count_leaves(model_7)
leaf_model_8 = count_leaves(model_8)
leaf_model_9 = count_leaves(model_9)

print "Complexity Model 7 : ",leaf_model_7

print "Complexity Model 8 : ",leaf_model_8

print "Complexity Model 9 : ",leaf_model_9

Complexity Model 7 :  41
Complexity Model 8 :  19
Complexity Model 9 :  1


# Model 8